In [1]:
import sys

import seaborn
from tensorflow.keras import datasets, layers, models, losses
import numpy as np
from PIL import Image
import tensorflow as tf
import torchvision as tv
import matplotlib.pyplot as plt
from livelossplot import PlotLossesKeras
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix
import pickle

sys.path.append('../../')

from core.datasets import get_ds, open_f
from core.make_answer import make_ans_file 
from core.ensembles import get_bagging_pred, print_bagging_ensemble_statistic, print_models_statistic, get_ensemble_modelbase, get_modeset, get_raw_bagging_pred


C:\Users\Salam\AppData\Roaming\Python\Python310\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


- Продолжение экспериментов с ансамблями

In [2]:
transformer = tv.transforms.Compose([
    tv.transforms.ColorJitter(brightness=.2, hue=0.2, contrast=0.5),
    tv.transforms.RandomAffine(degrees=(-10, 10), translate=(0, 0.1), scale=(0.85, 1)),
    tv.transforms.RandomPerspective(distortion_scale=0.2, p=0.7),
])

In [3]:
acc_test_ds = open_f("smote_data_train_5000", back=2)

In [4]:
acc_test_ds_y = acc_test_ds["labels"]
acc_test_ds_x = acc_test_ds["images"] / 255.

In [5]:
train_ds, val_ds, test_ds = get_ds('repaired_data_train', 'repaired_data_test', transform=transformer, one_hot=True, back=2)
val_ds_x = np.array([i[0] for i in val_ds.unbatch().as_numpy_iterator()])
val_ds_y = np.array([i[1] for i in val_ds.unbatch().as_numpy_iterator()])
val_ds_y = np.argmax(val_ds_y, axis=1)

In [ ]:
model_1 = tf.keras.models.load_model(f'../checkpoints/model_g_1_categorical_accuracy.h5')
model_3 = tf.keras.models.load_model(f'../checkpoints/model_g_3_avg_categorical_accuracy.h5')
model_smote = tf.keras.models.load_model(f'../checkpoints/model_s_smote_avg_categorical_accuracy.h5')
model_c = tf.keras.models.load_model(f'../checkpoints/model_C_EN_1_categorical_accuracy.h5')
model_8 = tf.keras.models.load_model(f'../checkpoints/model_g_8_avg_categorical_accuracy.h5')
model_16 = tf.keras.models.load_model(f'../checkpoints/model_g_16_avg_categorical_accuracy.h5')
model_17 = tf.keras.models.load_model(f'../checkpoints/model_g_17_avg_categorical_accuracy.h5')
model_g_11 = tf.keras.models.load_model(f'../checkpoints/model_g_11_avg_f1_score.h5')
model_s_6 = tf.keras.models.load_model(f'../checkpoints/model_s_6_avg_f1_score.h5')

In [7]:
ova_0 = tf.keras.models.load_model(f'../checkpoints/model_zero_avg_categorical_accuracy.h5')
ova_1 = tf.keras.models.load_model(f'../checkpoints/model_one_avg_categorical_accuracy.h5')
ova_2 = tf.keras.models.load_model(f'../checkpoints/model_two_avg_categorical_accuracy.h5')
ova_3 = tf.keras.models.load_model(f'../checkpoints/model_three_avg_categorical_accuracy.h5')
ova_4 = tf.keras.models.load_model(f'../checkpoints/model_four_avg_categorical_accuracy.h5')
ova_5 = tf.keras.models.load_model(f'../checkpoints/model_five_avg_categorical_accuracy.h5')
ova_6 = tf.keras.models.load_model(f'../checkpoints/model_six_avg_categorical_accuracy.h5')
ova_7 = tf.keras.models.load_model(f'../checkpoints/model_seven_avg_categorical_accuracy.h5')
ova_8 = tf.keras.models.load_model(f'../checkpoints/model_eight_avg_categorical_accuracy.h5')
ova_9 = tf.keras.models.load_model(f'../checkpoints/model_nine_avg_categorical_accuracy.h5')

ova = {0:ova_0, 1:ova_1, 2:ova_2, 3:ova_3, 4:ova_4, 5:ova_5,  6:ova_6, 7:ova_7, 8:ova_8, 9:ova_9}

In [224]:
predict_g11 = pickle.load(open("../checkpoint_answers/model_g_11_avg_categorical_accuracy.h5", 'rb'))
predict_s6 = pickle.load(open("../checkpoint_answers/model_s_6_avg_categorical_accuracy.h5", 'rb'))
predict_s5 = pickle.load(open("../checkpoint_answers/model_s_5_categorical_accuracy.h5", 'rb'))

raw_pred_test = (predict_g11 + predict_s6 + predict_s5) / 3

In [225]:
def min_max_scaler(x):
    min_x = min(x)
    max_x = max(x)
    result = []
    for i in x:
        result.append((i - min_x)/(max_x-min_x))
    return np.array(result)

# Получить первый/второй/третий по страшинству предикт. (максимальные логиты).
def get_ordinal_pred(y_logits, i):
    work = list(y_logits.copy())
    work.sort(reverse=True)
    return list(y_logits).index(work[i])

In [16]:
ova_test_preds = {}
for i in range(10):
    ova_test_preds[i] = ova[i].predict(test_ds, verbose=True)

final_ova_test_preds = np.full((len(test_ds), 10), 0.)
for i in range(len(test_ds)):
    l = np.zeros(10)
    for j in range(10):
        l[j] = ova_test_preds[j][i]
    final_ova_test_preds[i] = min_max_scaler(l)

final_ova_test_preds_witout_scaler= np.full((len(test_ds), 10), 0.)
for i in range(len(test_ds)):
    l = np.zeros(10)
    for j in range(10):
        l[j] = ova_test_preds[j][i]
    final_ova_test_preds_witout_scaler[i] = l.copy()

782/782 [==============================] - 37s 47ms/step


In [261]:
final_activated_ova= np.full((len(test_ds), 10), 0.)
for i in range(len(test_ds)):
    l = np.zeros(10)
    for j in range(10):
        l[j] = ova_test_preds[j][i]
        if l[j] < 0:
            l[j] = 0            
    final_activated_ova[i] = l.copy()

- Вариант "Apha Hybrid": Было выявлено, что так или иначе, когда модель ошибается, правильный ответ стоит у неё на первом или втором по размеру логита месте. Таким образом, если добавить 
некие дополнительные фильтры, то можно частично исключить эти ошибки. "Alpa Hybrid" представляет собой постобработку предиктов ансамбля типа bagging. В качестве фильтров используется 10 моделей "one vs all", которые определяют принадлежит ли число указанному классу. 

In [226]:
# ALPHA HYBRID TEST

changes_count = 0
undo_count = 0
cooked_pred = []
for i in range(len(raw_pred_test)):
    raw_y = raw_pred_test[i]
    first_y_pred = get_ordinal_pred(raw_y, 0)
    if ova_test_preds[first_y_pred][i] >= 0:
        cooked_pred.append(first_y_pred)
        continue
    changes_count += 1

    second_y_pred = get_ordinal_pred(raw_y, 1)
    if ova_test_preds[second_y_pred][i] >= 0:
        cooked_pred.append(second_y_pred)
        continue

    third_y_pred = get_ordinal_pred(raw_y, 2)
    if ova_test_preds[third_y_pred][i] >= 0:
        cooked_pred.append(third_y_pred)
        continue

    changes_count-=1
    undo_count +=1
    cooked_pred.append(first_y_pred)

print(f"TOTAL: {changes_count} changes, {undo_count} undo")
wunderwafel_ensy_alphahybrid = np.array(cooked_pred)

TOTAL: 327 changes, 616 undo


In [37]:
import pandas as pd

ensemble_model_name = "ENSY_C5_APHAHYBRID_wunderwafel"
ans = pd.DataFrame({'Id': np.arange(wunderwafel_ensy_alphahybrid.shape[0]), 'Category': wunderwafel_ensy_alphahybrid})
ans.to_csv(f"../answers/{ensemble_model_name}.csv", index=False)

- Вариант "Beta Hybrid": тут используется иной подход. Берётся 10 логитов с ova-моделей. Предикт каждой ova-модели относительно какого-то числа. Далее к этому массиву применяется MinMaxScaler и этот массив
умножается на массив суммы логитов с ансамбля. С полученного результата берётся максимальный логит. 

In [227]:
# BETA HYBRID TEST
cooked_pred = []
nothing_changed = 0
for i in range(len(raw_pred_test)):
    if np.argmax(raw_pred_test[i]) == np.argmax(final_ova_test_preds[i]):
        cooked_pred.append(np.argmax(raw_pred_test[i]))
        nothing_changed+=1
        continue
    raw_y = raw_pred_test[i]
    w = final_ova_test_preds[i] * raw_y
    cooked_pred.append(np.argmax(w))


changed_count = np.sum(cooked_pred != np.array([np.argmax(k) for k in raw_pred_test]))
tried_to_change = len(test_ds) - nothing_changed - changed_count
print(f"TOTAL: {changed_count} changes, {tried_to_change} try change, {nothing_changed} not changed")
wunderwafel_ensy_betahybrid = np.array(cooked_pred)

TOTAL: 164 changes, 763 try change, 24073 not changed


In [276]:
import pandas as pd

ensemble_model_name = "1ENSY_C5_BETAHYBRID_wunderwafel"
ans = pd.DataFrame({'Id': np.arange(wunderwafel_ensy_betahybrid.shape[0]), 'Category': wunderwafel_ensy_betahybrid})
ans.to_csv(f"../answers/{ensemble_model_name}.csv", index=False)

- Вариант "Gamma Hybrid": похож на "Beta Hybrid", однако не применяется MinMaxScaler к ova-предиктам и вместо умножения прибавление к сумме логитов с ансамбля.

In [652]:
# GAMMA HYBRID TEST
cooked_pred = []
nothing_changed = 0


# отладка
tried_to_change_l = []


for i in range(len(raw_pred_test)):
    if np.argmax(raw_pred_test[i]) == np.argmax(final_ova_test_preds_witout_scaler[i]):
        cooked_pred.append(np.argmax(raw_pred_test[i]))
        nothing_changed+=1
        continue
    raw_y = raw_pred_test[i]
    w = final_ova_test_preds_witout_scaler[i]/3 + raw_y
    tried_to_change_l.append(i)
    cooked_pred.append(np.argmax(w))


changed_count = np.sum(cooked_pred != np.array([np.argmax(k) for k in raw_pred_test]))
tried_to_change = len(test_ds) - nothing_changed - changed_count
print(f"TOTAL: {changed_count} changes, {tried_to_change} try change, {nothing_changed} not changed")
wunderwafel_ensy_gammahybrid = np.array(cooked_pred)

TOTAL: 321 changes, 606 try change, 24073 not changed


In [249]:
import pandas as pd

ensemble_model_name = "ENSY_C5_GAMMAHYBRID_wunderwafel"
ans = pd.DataFrame({'Id': np.arange(wunderwafel_ensy_gammahybrid.shape[0]), 'Category': wunderwafel_ensy_gammahybrid})
ans.to_csv(f"../answers/{ensemble_model_name}.csv", index=False)

Наиболее хорошие результаты показывают Beta и Gamma гибриды

- Серия Hybrid первой смогла пробить accuracy 0.96, однако позже, вышла на потолок своих структурных возможностей. Более поздние ансамбли с новыми, более качественными моделями, показывали результы до обработки гибридами лучше, чем после. Причина заключается в том, что данная серия структур зависима от ova-моделей, а они в отличии от стандартных моделей не переписывались и не улучшались. По итогу, когда разница между моделями из ансамбля и ova-моделями стала слишком большой, ova-модели стали баластом, который тянул accuracy вниз, а не повышал, как раньше. Потенциально, если переписать ova-модели на том же уровне, на котором сейчас написаны последние модели, то потолок 0.97 можно будет пробить без особых проблем, но это слишком затратно по времени, выгоднее вложиться в обычные модели.

Вывод: серия "Hybrid" сыграла важную роль в обеспечении преимущества alt+f4, однако устарела и перестала быть эффективной после потолка accuracy 0.963.